前言
时间序列数据，顾名思义，是一种随着时间改变的数据。例如，
24小时气温数据，
一个月的产品价格数据，
某一公司股票价格年度数据。
。。。。。。
高级深度学习模型，比如长短期记忆网络（LSTM），能够捕获到时间序列数据中的变化模式，进而能够预测数据的未来趋势。本文中，我们将使用pytorch这个深度学习库，来实现利用LSTM算法对时间序列数据进行预测。

在开始讲述之前，我们先导入必要的库，

In [3]:
import torch
import torch.nn as nn
 
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 
%matplotlib inline

数据集
我们将使用Seaborn库的内建数据集。让我们打印一下Seaborn的所有内建数据库：

In [4]:
sns.get_dataset_names()

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'dowjones',
 'exercise',
 'flights',
 'fmri',
 'geyser',
 'glue',
 'healthexp',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'seaice',
 'taxis',
 'tips',
 'titanic']

我们将使用flights数据集。可以用如下代码导入：

In [5]:
flight_data = sns.load_dataset("flights")
flight_data.head()

,year,month,passengers
0,1949,Jan,112
1,1949,Feb,118
2,1949,Mar,132
3,1949,Apr,129
4,1949,May,121


数据集有3列：年，月和乘客数量。乘客数量一列描述了单月内航班乘客总数。

数据集的形状：



In [6]:
print(flight_data)
flight_data.shape


     year month  passengers
0    1949   Jan         112
1    1949   Feb         118
2    1949   Mar         132
3    1949   Apr         129
4    1949   May         121
..    ...   ...         ...
139  1960   Aug         606
140  1960   Sep         508
141  1960   Oct         461
142  1960   Nov         390
143  1960   Dec         432

[144 rows x 3 columns]


(144, 3)

可以看到，一共有144行和3列数据，即数据集包含12年的乘客记录。 我们的任务是利用前132个月的数据预测最后12个月乘客数。也就是说前132个月的数据用作训练，最后12个月的数据用作验证以评估模型。

让我们来绘制每个月乘客出行的频率，

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 15
fig_size[1] = 5
plt.rcParams["figure.figsize"] = fig_size
 
plt.title('Month vs Passenger')
plt.ylabel('Total Passengers')
plt.xlabel('Months')
plt.grid(True)
plt.autoscale(axis='x',tight=True)
plt.plot(flight_data['passengers'])

如图所示，多年来，乘飞机旅行的平均人数增加了。一年内旅行的乘客数量是波动的，这是有道理的，因为在夏季或冬季休假期间，旅行的乘客数量比一年中的其他时间增加。

构建LSTM需要的样本集
我们先看一下数据列中数据的类型，　

In [7]:
flight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   year        144 non-null    int64   
 1   month       144 non-null    category
 2   passengers  144 non-null    int64   
dtypes: category(1), int64(2)
memory usage: 2.9 KB


 数据处理的第一步是将乘客数量一列的数据类型转换为float，

In [8]:
all_data = flight_data['passengers'].values.astype(float)

现在，如果你打印all_data这个numpy数组，你应该看到以下float类型的值。　　

接下来，我们将把我们的数据集分为训练集和测试集。LSTM算法将在训练集上进行训练。然后，该模型将被用来对测试集进行预测。预测结果将与测试集的实际值进行比较，以评估训练模型的性能。前132条记录将被用来训练模型，最后12条记录将被用作测试集。下面的代码将数据分为训练集和测试集。

In [9]:
test_data_size = 12
 
train_data = all_data[:-test_data_size]
test_data = all_data[-test_data_size:]

让我们打印一下训练集和测试集的样本个数：

我们的数据集目前还没有被归一化（normalization）。最初几年的乘客总数与后来几年的乘客总数相比要少得多。对于时间序列预测来说，将数据归一化是非常重要的。我们将对数据集进行最小/最大缩放，使数据在一定的最小值和最大值范围内归一化。我们将使用sklearn.preprocessing模块中的MinMaxScaler类来缩放我们的数据。下面的代码使用最小/最大缩放器对我们的数据进行归一化处理，最小值和最大值分别为-1和1。

In [10]:
from sklearn.preprocessing import MinMaxScaler
 
scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data .reshape(-1, 1))

In [11]:
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)


In [12]:
train_window = 12

In [13]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [14]:
train_inout_seq = create_inout_sequences(train_data_normalized, train_window)

In [15]:
train_inout_seq[:5]


[(tensor([-0.9648, -0.9385, -0.8769, -0.8901, -0.9253, -0.8637, -0.8066, -0.8066,
          -0.8593, -0.9341, -1.0000, -0.9385]),
  tensor([-0.9516])),
 (tensor([-0.9385, -0.8769, -0.8901, -0.9253, -0.8637, -0.8066, -0.8066, -0.8593,
          -0.9341, -1.0000, -0.9385, -0.9516]),
  tensor([-0.9033])),
 (tensor([-0.8769, -0.8901, -0.9253, -0.8637, -0.8066, -0.8066, -0.8593, -0.9341,
          -1.0000, -0.9385, -0.9516, -0.9033]),
  tensor([-0.8374])),
 (tensor([-0.8901, -0.9253, -0.8637, -0.8066, -0.8066, -0.8593, -0.9341, -1.0000,
          -0.9385, -0.9516, -0.9033, -0.8374]),
  tensor([-0.8637])),
 (tensor([-0.9253, -0.8637, -0.8066, -0.8066, -0.8593, -0.9341, -1.0000, -0.9385,
          -0.9516, -0.9033, -0.8374, -0.8637]),
  tensor([-0.9077]))]

In [17]:
class LSTM(nn.Module):
    def __init__(self,input_size=1,hidden_layer_size=100,output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size,hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size,output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),torch.zeros(1,1,self.hidden_layer_size))
 
    def forward(self,input_seq):
        lstm_out,self.hidden_cell = self.lstm(input_seq.view(len(input_seq),1,-1), self.hidden_cell)
        predictions.shape:[L,1]
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        predictions[-1].shape:[1]
        return predictions[-1]

In [18]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [19]:
print(model)

LSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [20]:
epochs = 150
 
for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
 
        y_pred = model(seq)
 
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
 
    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')
        
print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

UnboundLocalError: local variable 'predictions' referenced before assignment

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.num_directions = 1 # 单向LSTM
        self.batch_size = batch_size
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input_seq):
        h_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size).to(device)
        c_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size).to(device)
        seq_len = input_seq.shape[1] # (5, 30)
        # input(batch_size, seq_len, input_size)
        input_seq = input_seq.view(self.batch_size, seq_len, 1)  # (5, 30, 1)
        # output(batch_size, seq_len, num_directions * hidden_size)
        output, _ = self.lstm(input_seq, (h_0, c_0)) # output(5, 30, 64)
        output = output.contiguous().view(self.batch_size * seq_len, self.hidden_size) # (5 * 30, 64)
        pred = self.linear(output) # pred(150, 1)
        pred = pred.view(self.batch_size, seq_len, -1) # (5, 30, 1)
        pred = pred[:, -1, :]  # (5, 1)
        return pred

In [ ]:
self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
self.linear = nn.Linear(self.hidden_size, self.output_size)

In [ ]:
我们加上具体的数字：

In [ ]:
self.lstm = nn.LSTM(self.input_size=1, self.hidden_size=64, self.num_layers=5, batch_first=True)
self.linear = nn.Linear(self.hidden_size=64, self.output_size=1)